In [1]:
import numpy as np    # Numeric and matrix computation
import pandas as pd   # Optional: good package for manipulating data 
import sklearn   # Package with learning algorithms implemented
from sklearn.model_selection import train_test_split, GridSearchCV # split
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score
#from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing      # Preprocessing

In [3]:
df = pd.read_csv("preprocessed_mean.csv");
df = pd.get_dummies(df, columns=["mode","key"])

targetCol = 'music_genre';
Y = df[targetCol]
X = df.drop([targetCol],axis=1)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=True, random_state=33)
scaler = preprocessing.Normalizer().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [4]:
cv = 10
clf1 = GaussianNB()
clf2 = KNeighborsClassifier(metric = 'manhattan', n_neighbors = 20, weights = 'distance', p = 3)
# scores = cross_val_score(clf2, X_train, Y_train, cv=cv, scoring='accuracy')
# print("Accuracy: %0.3f [%s]" % (scores.mean(), "Knn"))
clf3 = DecisionTreeClassifier(criterion='entropy', max_depth=8, min_samples_leaf=6, min_samples_split=15)
clf4 = DecisionTreeClassifier()
for clf, label in zip([clf1,clf2, clf3, clf4], ['Naive Bayes','Knn', 'Dec. Tree with param', 'Dec. Tree']):
    scores = cross_val_score(clf, X_train, Y_train, cv=cv, scoring='accuracy')
    print("Accuracy: %0.3f [%s]" % (scores.mean(), label))

Accuracy: 0.617 [Naive Bayes]
Accuracy: 0.710 [Knn]
Accuracy: 0.714 [Dec. Tree with param]
Accuracy: 0.769 [Dec. Tree]


In [5]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

eclf = VotingClassifier(estimators=[('nb', clf1), ('knn3', clf2), ('dt param', clf3), ('dt', clf4)], voting='hard')
scores = cross_val_score(eclf, X_train, Y_train, cv=cv, scoring='accuracy')
print("Accuracy: %0.3f [%s]" % (scores.mean() , "Majority Voting"))

Accuracy: 0.783 [Majority Voting]


In [6]:
eclf = VotingClassifier(estimators=[('nb', clf1), ('knn3', clf2), ('dt param', clf3),('dt', clf4) ],voting='soft', weights=[1,2,2,2])
scores = cross_val_score(eclf, X_train, Y_train, cv=cv, scoring='accuracy')
print("Accuracy: %0.3f [%s]" % (scores.mean(), "Weighted Voting"))

Accuracy: 0.802 [Weighted Voting]


In [7]:
eclf.fit(X_train,Y_train)
pred = eclf.predict(X_test);
print(sklearn.metrics.classification_report(Y_test,pred))

              precision    recall  f1-score   support

   Classical       0.93      0.90      0.91       955
  Electronic       0.83      0.81      0.82      1057
     Hip-Hop       0.74      0.74      0.74       995
        Rock       0.71      0.75      0.73       993

    accuracy                           0.80      4000
   macro avg       0.80      0.80      0.80      4000
weighted avg       0.80      0.80      0.80      4000

